In [1]:
from PIL import Image
from tqdm import tqdm
import binascii
import glob
import os

import numpy as np

import keras.preprocessing.image

dataCnt = 0
dataNum = 9000
width = 100
height = 100
channels = 3
dataset = np.zeros(shape=(dataNum, height, width, channels), dtype=np.float32)

In [2]:
def main():
    global dataCnt
    malware_path = './bytes'
    malware_categories = os.listdir(malware_path+'/')

    dataCnt = 0
    cnt = 0

    for malware_family in tqdm(malware_categories):
        malware_files = glob.glob(malware_path+'/'+malware_family+'/*.txt')
        for malware_file in tqdm(malware_files):
            if not malware_file:
                continue
            try:
                f = open(malware_file, 'r')
                code = f.readlines()
                one_line = ''
                for line in code:
                    line = line.replace("+", "")
                    line = line.replace("\n", "")

                    if len(line) != 2:
                        print(str(len(line)) +"### " + line)
                    one_line = one_line + line

                BinarySrcCode = binascii.unhexlify(one_line)
                code_len = len(BinarySrcCode)
                padLen = width * height * 3 - code_len

                if code_len == 0:
                    continue
                if padLen < 0:
                    print('file is over')
                    cnt += 1
                    continue

                zeroPad = bytes(padLen)
                image = bytearray(BinarySrcCode) + bytes(zeroPad)
                img = Image.frombytes("RGB", (height, width), bytes(list(image)))
                img.save("./malware_images/"+ malware_family + "/" + malware_file[17:-4] + ".PNG", 'PNG')
                dataCnt += 1
            except Exception as ex:
                print(malware_file)
                print("Dd")
                print(ex)
                return

    print("num of exceeded sol : ", cnt)
    print("num of total vul : ", dataCnt)

    return

main()

100%|██████████| 9/9 [00:53<00:00,  6.00s/it]

num of exceeded sol :  0
num of total vul :  2149


In [4]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

data_dir = "./malware_images/"
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_it = datagen.flow_from_directory(
    data_dir,
    class_mode='categorical',
    target_size=(100, 100),
    batch_size=32
)

val_it = validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(100, 100),
    batch_size=32,
    subset='validation'
)

cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100,100,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')])

cnn_model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

cnn_model.fit(train_it, epochs=10)
test_eval_result = cnn_model.evaluate(val_it)
print(test_eval_result)

converter = tf.lite.TFLiteConverter.from_keras_model(cnn_model)
tflite_model = converter.convert()

with open('./keras_model_cnn_malware.tflite', 'wb') as f:
    f.write(tflite_model)

Found 2149 images belonging to 9 classes.
Found 426 images belonging to 9 classes.
Epoch 1/10
68/68 [==============================] - 24s 345ms/step - loss: 1.2995 - accuracy: 0.5361
Epoch 2/10
68/68 [==============================] - 22s 323ms/step - loss: 0.8720 - accuracy: 0.6864
Epoch 3/10
68/68 [==============================] - 22s 326ms/step - loss: 0.5655 - accuracy: 0.8148
Epoch 4/10
68/68 [==============================] - 22s 329ms/step - loss: 0.3954 - accuracy: 0.8683
Epoch 5/10
68/68 [==============================] - 22s 318ms/step - loss: 0.3234 - accuracy: 0.8944
Epoch 6/10
68/68 [==============================] - 23s 333ms/step - loss: 0.2157 - accuracy: 0.9325
Epoch 7/10
68/68 [==============================] - 24s 349ms/step - loss: 0.1586 - accuracy: 0.9544
Epoch 8/10
68/68 [==============================] - 22s 319ms/step - loss: 0.1149 - accuracy: 0.9679
Epoch 9/10
68/68 [==============================] - 24s 350ms/step - loss: 0.0887 - accuracy: 0.9702
Epoch 10

INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmpww90l9je\assets


INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmpww90l9je\assets
